# A widget for interactive analysis of a matchup

## Input parameters.

In [41]:
TEAM_1 = 'J.Storm'
TEAM_2 = 'Team Plus'

FORCE_REFIT = True

LOGISTIC_SCALE = 3.0
COV_FUNC_NAME = 'exponential'
COV_FUNC_KWARGS = {'scale': 1.25 * 365 * 24 * 60 * 60 * 1000}
RADI_PRIOR_SD = 3.0

In [42]:
TEAM_1 = TEAM_1.lower()
TEAM_2 = TEAM_2.lower()

## Load libraries.

In [3]:
import os
while 'src' not in os.listdir():
    os.chdir('..')
os.getcwd()

'/Users/yl3/github/dota2'

In [4]:
import dill
import pandas as pd

import src.load
import src.models.gp
import src.stats

## Load data.

In [25]:
def load_premium_prof_matches():
    prem_matches = src.load.fetch_matches()
    prof_matches = src.load.fetch_matches('professional')
    matches = src.load.MatchDF(pd.concat([prem_matches.df, prof_matches.df]))
    return matches

In [6]:
def _match_date(matches):
    """Compute the match dates of each team by team name.

    Args:
        matches (load.MatchDF): The match data.
    """
    radi_teams = matches.df[['radiant_name', 'radiant_valveId',
                             'radiant_nicknames', 'radiant_players',
                             'startDate']]
    dire_teams = matches.df[['dire_name', 'dire_valveId',
                             'dire_nicknames', 'dire_players',
                             'startDate']]
    colnames = ['team_name', 'team_id', 'player_names', 'player_ids',
                'startDate']
    radi_teams.columns = colnames
    dire_teams.columns = colnames
    teams_by_date = (pd.concat([radi_teams, dire_teams])
                     .sort_values('startDate')
                     .reset_index())
    for colname in ('matchId', 'team_id'):
        teams_by_date[colname] = \
            teams_by_date[colname].astype(pd.Int64Dtype())
    teams_by_date['team_name'] = teams_by_date['team_name'].str.lower()
    teams_by_date.index = teams_by_date['team_name']
    return teams_by_date

In [26]:
serialised_model_file = 'interactive/latest_fitted_gp_model.dill'
if FORCE_REFIT or not os.path.isfile(serialised_model_file):
    matches = load_premium_prof_matches()
    gp_model = src.models.gp.SkillsGPMAP.from_match_df(
        matches,
        cov_func_name=COV_FUNC_NAME,
        cov_func_kwargs=COV_FUNC_KWARGS,
        radi_prior_sd=RADI_PRIOR_SD,
        logistic_scale=LOGISTIC_SCALE
    )
    gp_model.fit()
    display(gp_model.fitted)
    match_pred = src.stats.MatchPred(
        matches,
        gp_model.fitted_pred_df(),
        gp_model.logistic_scale,
        gp_model.fitted_skills_mat())
    with open(serialised_model_file, 'wb') as fh:
        dill.dump(match_pred, fh)
else:
    with open(serialised_model_file, 'rb') as fh:
        match_pred = dill.load(fh)

     fun: -248442.98370921946
     jac: array([ 8.09542501e-07, -7.04042368e-07, -9.20713793e-07, ...,
       -1.83662367e-07,  2.26485291e-06, -2.44725543e-05])
 message: 'Optimization terminated successfully.'
    nfev: 17
    nhev: 15106
     nit: 16
    njev: 32
  status: 0
 success: True
       x: array([ 0.11461389,  0.11431448,  0.11438803, ..., -0.03285866,
       -0.14133105,  0.33703911])

### Compute team selectors.

In [43]:
loc1 = ((match_pred.df.radiant_name.str.lower() == TEAM_1)
        | (match_pred.df.dire_name.str.lower() == TEAM_1))
loc2 = ((match_pred.df.radiant_name.str.lower() == TEAM_2)
        | (match_pred.df.dire_name.str.lower() == TEAM_2))
team1_id = match_pred.teams.index[match_pred.teams.str.lower() == TEAM_1][-1]
team2_id = match_pred.teams.index[match_pred.teams.str.lower() == TEAM_2][-1]

# Get the most recent roster of both teams.
match_dates = _match_date(match_pred)
team_1_players = match_dates.loc[
    match_dates.team_id == team1_id, 'player_ids'].iloc[-1]
team_2_players = match_dates.loc[
    match_dates.team_id == team2_id, 'player_ids'].iloc[-1]
    
display(match_pred.teams[match_pred.teams.str.lower() == TEAM_1])
display(match_pred.teams[match_pred.teams.str.lower() == TEAM_2])
print(f"{sum(loc1)} matches for team 1.")
print(f"{sum(loc2)} matches for team 2.")

6288801    J.Storm
7409177    J.Storm
dtype: object

7408305    Team Plus
dtype: object

193 matches for team 1.
6 matches for team 2.


## Actual interactive plots

### Previous matches of both teams.

In [28]:
match_pred.df.loc[loc1].tail(20)

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,seriesId,duration,radiant_valveId,...,dire_players,league_id,startTimestamp,series_start_time,match_i_in_series,pred_win_prob,radi_skill,dire_skill,pred_win_prob_unknown_side,radi_adv
matchId,,,,,,,,,,,,,,,,,,,,,
5043533011,2019-09-27 19:06:37,ESL One Hamburg 2019 powered by Intel,Fighting Pepegas,Black Sheep!,True,"(Sneyking, MoonMeander, Aui_2000, EternaLEnVy,...","(no, Scourge McDuck, Legend, Husky, Oscar`)",371278,1792,7391166,...,"(59752811, 85367644, 98462965, 110448679, 3526...",11275,1569611197000,1569611197000,0,0.731559,2.028551,-0.642043,0.708388,0.337039
5043593443,2019-09-27 20:03:08,ESL One Hamburg 2019 powered by Intel,Fighting Pepegas,Black Sheep!,True,"(Sneyking, MoonMeander, Aui_2000, EternaLEnVy,...","(no, Scourge McDuck, Legend, Husky, Oscar`)",371278,1677,7391166,...,"(59752811, 85367644, 98462965, 110448679, 3526...",11275,1569614588000,1569611197000,1,0.731554,2.028453,-0.642065,0.708383,0.337039


In [29]:
match_pred.df.loc[loc2].tail(20)

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,seriesId,duration,radiant_valveId,...,dire_players,league_id,startTimestamp,series_start_time,match_i_in_series,pred_win_prob,radi_skill,dire_skill,pred_win_prob_unknown_side,radi_adv
matchId,,,,,,,,,,,,,,,,,,,,,
4904261164,2019-07-15 00:19:24,The International 2019,Newbee,J.Storm,True,"(pieliedie, Sneyking, MSS, YawaR, CCnC)","(Moo, Resolut1on, Fear, 1437, nine)",348164,1901,6214538,...,"(84429681, 86725175, 87177591, 87196890, 94786...",10749,1563149964000,1563142190000,2,0.750148,3.924082,0.962912,0.727937,0.337039
4904311356,2019-07-15 01:30:53,The International 2019,J.Storm,Newbee,False,"(Moo, Resolut1on, Fear, 1437, nine)","(pieliedie, Sneyking, MSS, YawaR, CCnC)",348164,1964,6288801,...,"(6922000, 10366616, 86726887, 108452107, 22166...",10749,1563154253000,1563142190000,3,0.294235,0.962054,3.923798,0.272026,0.337039
4925176841,2019-07-25 16:05:44,DOTA Summit 10,Alliance,J.Storm,True,"(iNSaNiA, Boxi, qojqva, miCKe, Taiga)","(Moo, Resolut1on, Fear, 1437, nine)",351515,2709,111474,...,"(84429681, 86725175, 87177591, 87196890, 94786...",11162,1564070744000,1564070744000,0,0.775872,4.189984,0.801708,0.755138,0.337039
4925317287,2019-07-25 17:28:34,DOTA Summit 10,Alliance,J.Storm,True,"(iNSaNiA, Boxi, qojqva, miCKe, Taiga)","(Moo, Resolut1on, Fear, 1437, nine)",351515,2143,111474,...,"(84429681, 86725175, 87177591, 87196890, 94786...",11162,1564075714000,1564070744000,1,0.775922,4.190076,0.800934,0.755191,0.337039
4925450824,2019-07-25 19:00:01,DOTA Summit 10,beastcoast,J.Storm,False,"(Brax, ixmike88, Ryoya, M-GOD, MoOz)","(Moo, Resolut1on, Fear, 1437, nine)",351576,2952,7079109,...,"(84429681, 86725175, 87177591, 87196890, 94786...",11162,1564081201000,1564081201000,0,0.528779,0.808876,0.800185,0.500722,0.337039
4925548100,2019-07-25 20:20:13,DOTA Summit 10,J.Storm,beastcoast,False,"(Moo, Resolut1on, Fear, 1437, nine)","(Brax, ixmike88, Ryoya, M-GOD, MoOz)",351576,2793,6288801,...,"(31818853, 86715129, 115141430, 131706718, 349...",11162,1564086013000,1564081201000,1,0.527220,0.799310,0.809386,0.499163,0.337039
4925748555,2019-07-26 00:17:23,DOTA Summit 10,paiN Gaming,J.Storm,True,"(Stormwind, LESLÃO, hFn, Thiolicor, Unknown)","(Moo, Resolut1on, Fear, 1437, nine)",351637,2355,67,...,"(84429681, 86725175, 87177591, 87196890, 94786...",11162,1564100243000,1564100243000,0,0.685855,2.802752,0.797362,0.660704,0.337039
4925807866,2019-07-26 01:25:54,DOTA Summit 10,paiN Gaming,J.Storm,True,"(Stormwind, LESLÃO, hFn, Thiolicor, Unknown)","(Moo, Resolut1on, Fear, 1437, nine)",351637,2156,67,...,"(84429681, 86725175, 87177591, 87196890, 94786...",11162,1564104354000,1564100243000,1,0.685883,2.802699,0.796908,0.660734,0.337039
4927545339,2019-07-26 18:22:31,DOTA Summit 10,Team Serenity,J.Storm,True,"(Roddgee, XCJ, Zyd, Ahjit, Pyw)","(Moo, Resolut1on, Fear, 1437, nine)",351942,2267,5066616,...,"(84429681, 86725175, 87177591, 87196890, 94786...",11162,1564165351000,1564165351000,0,0.477021,0.178814,0.791797,0.449254,0.337039


### Team skills

In [44]:
match_pred.team_skills_plot(
    match_pred.teams.index[
        match_pred.teams.str.lower().isin([TEAM_1, TEAM_2])
    ])

### Player skills

In [45]:
match_pred.player_skills_plot(team_1_players + team_2_players)